In [ ]:
import idom

import panel as pn

pn.extension()

**WARNING**: Currently Panel only support idom version 0.24. We are working on supporting later version as part of the [idom-bokeh project](https://github.com/idom-team/idom-bokeh/).

The ``IDOM`` pane renders any [IDOM component](http://idom-docs.herokuapp.com/) both in the notebook and in a deployed server. IDOM defines an API for defining and controlling interactive HTML components directly from Python. Note that in the notebook the IDOM support for loading external modules relies on Panel's Jupyter serverextension. To check if this is enabled you can run:

    jupyter serverextension list
    
You should see:

    panel.io.jupyter_server_extension  enabled
    - Validating...
      panel.io.jupyter_server_extension  OK

If you don't see this but have installed panel you can manually enable the server extension with:

    jupyter serverextension enable --sys-prefix panel.io.jupyter_server_extension
    
which will install it where your Python is installed or to place it in the Jupyter config in your home directory:

    jupyter serverextension enable --py panel

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``object``** (object): The IDOM component being displayed

##### Display

* **``default_layout``** (pn.layout.Panel, default=Row): Layout to wrap the plot and widgets in

___

The `panel` function will automatically convert any ``idom.component`` into a displayable panel, while keeping all of its interactive features:

In [ ]:
@idom.component
def ClickCount():
    count, set_count = idom.hooks.use_state(0)

    return idom.html.button(
        {"onClick": lambda event: set_count(count + 1)},
        [f"Click count: {count}"],
    )

pn.pane.IDOM(ClickCount, width=300)

This makes it possible to generate even complex interactive components directly from Python, e.g. here we will create a ToDo list:

In [ ]:
@idom.component
def Todo():
    items, set_items = idom.hooks.use_state([])

    async def add_new_task(event):
        if event["key"] == "Enter":
            set_items(items + [event["value"]])

    tasks = []

    for index, text in enumerate(items):

        async def remove_task(event, index=index):
            set_items(items[:index] + items[index + 1 :])

        task_text = idom.html.td(idom.html.p(text))
        delete_button = idom.html.td({"onClick": remove_task}, idom.html.button(["x"]))
        tasks.append(idom.html.tr(task_text, delete_button))

    task_input = idom.html.input({"onKeyDown": add_new_task})
    task_table = idom.html.table(tasks)

    return idom.html.div(task_input, task_table)

pn.pane.IDOM(Todo)

If you have a live server backing your session, whether that is a notebook server or a Bokeh/Panel server deployment you can also use external Javascript components which will be compiled before first use. See the [idom documentation](https://idom-docs.herokuapp.com/docs/javascript-components.html) for more details on using external components. Note that to ensure that the JS modules are installed in the correct place you should use `pn.pane.IDOM.install` rather than simply using `idom.install`:

In [ ]:
victory = pn.pane.IDOM.install("victory", fallback="loading...")

victory_com = idom.component(
    lambda: victory.VictoryBar({"style": {"parent": {"width": "500px"}}}),
)

pn.pane.IDOM(victory_com)

In order to work with Panel components seamlessly the `IDOM` pane also provides a `use_param` method which allows us to use the current parameter value much like we would when using `pn.depends`:

In [ ]:
aw = pn.widgets.IntSlider(name='a', start=0, end=20, value=1)
bw = pn.widgets.IntSlider(name='b', start=0, end=20, value=1)

@idom.component
def view():
    a = pn.pane.IDOM.use_param(aw)
    b = pn.pane.IDOM.use_param(bw.param.value) # equivalent to passing in the widget
    return idom.html.div({}, f'{a}+{b}={a+b}')

pn.Row(aw, bw, view)